# Fun with Neural Nets

---

Below is a procedure for building a neural network to recognize handwritten digits.  The data is from Kaggle, and you will submit your results to Kaggle to test how well you did!

1. Load the training data (`train.csv`) from Kaggle
2. Setup X and y (feature matrix and target vector)
3. Split X and y into train and test subsets.
4. Preprocess your data

   - When dealing with image data, you need to normalize your `X` by dividing each value by the max value of a pixel (255).
   - Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix
   
5. Create your network.

   - Remember that for multi-class classification you need a softamx activation function on the output layer.
   - You may want to consider using regularization or dropout to improve performance.
   
6. Trian your network.
7. If you are unhappy with your model performance, try to tighten up your model by adding hidden layers, adding hidden layer units, chaning the activation functions on the hidden layers, etc.
8. Load in Kaggle's `test.csv`
9. Create your predictions (these should be numbers in the range 0-9).
10. Save your predictions and submit them to Kaggle.

---

For this lab, you should complete the above sequence of steps for _at least_ two of the three "configurations":

1. Using a `tensorflow` network
2. Using a `keras` "sequential" network
3. Using a `keras` convolutional network
4. Using a `tensorflow` convolutional network (we did _not_ cover this in class!)

In [1]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from keras.utils import to_categorical
from keras import backend
import numpy as np

C:\Users\tbaca\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\tbaca\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
df = pd.read_csv('./Data/train.csv')
submit = pd.read_csv('./Data/test.csv')

In [3]:
X = df.drop('label', axis = 1)/255

In [4]:
y = df['label']

In [5]:
y = to_categorical(y)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [7]:
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()

input_units = X_train.shape[1]
hidden_units = input_units

model.add(Dense(hidden_units, input_dim=input_units, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))

model.add(Dense(10, activation='softmax'))

from keras.optimizers import Adam

#model.compile(loss='mean_squared_error', optimizer='adam')

adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics =['accuracy'])

history = model.fit(X, y, 
          epochs=5)

Epoch 1/5
42000/42000 [==============================] - 18s 436us/step - loss: 0.2701 - acc: 0.9190
Epoch 2/5
42000/42000 [==============================] - 17s 410us/step - loss: 0.1076 - acc: 0.9680
Epoch 3/5
42000/42000 [==============================] - 17s 411us/step - loss: 0.0779 - acc: 0.9762
Epoch 4/5
42000/42000 [==============================] - 19s 455us/step - loss: 0.0579 - acc: 0.9825
Epoch 5/5
42000/42000 [==============================] - 20s 471us/step - loss: 0.0477 - acc: 0.9846


In [9]:
SeqNNpreds = model.predict(submit)

In [17]:
PREDS

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [11]:
import tensorflow as tf
X = tf.placeholder(dtype=tf.float32,shape = (None, X_train.shape[1]),name = 'X')
y = tf.placeholder(dtype = tf.float32, shape = (None), name = 'y')

hidden1 = tf.layers.dense(X,units = X_train.shape[1], name = 'hidden1', activation = tf.nn.relu)
hidden2 = tf.layers.dense(X,units = 200, name = 'hidden2', activation = tf.nn.relu)
hidden3 = tf.layers.dense(X,units = 100, name = 'hidden3', activation = tf.nn.relu)
hidden4 = tf.layers.dense(X,units = 50, name = 'hidden4', activation = tf.nn.relu)
hidden5 = tf.layers.dense(X,units = 25, name = 'hidden5', activation = tf.nn.relu)

y_hat = tf.layers.dense(hidden1, units=y_train.shape[1], activation=None)

loss = tf.losses.softmax_cross_entropy(y, y_hat)

optimizer = tf.train.AdamOptimizer(0.001)

training_op = optimizer.minimize(loss)

saver = tf.train.Saver()

In [12]:
from sklearn.metrics import accuracy_score

In [15]:
init = tf.global_variables_initializer()

test_err = []
train_err = []

with tf.Session() as sess:
    init.run()
    
    for epoch in range(50):
        sess.run(training_op,
                 feed_dict={X:X_train, y:y_train})
        
        train_loss = sess.run(loss, feed_dict={X:X_train, y:y_train})
        test_loss = sess.run(loss, feed_dict={X:X_test, y:y_test})
        test_err.append(test_loss)
        train_err.append(train_loss)
        if epoch % 10 == 0:
            print('epoch', epoch+1, train_loss, test_loss)
            
    saver.save(sess, './iris.ckpt')
        
    pred = sess.run(y_hat, feed_dict={X:submit})

epoch 1 1.9172049 1.9177629
epoch 11 0.5067511 0.5058806
epoch 21 0.33770096 0.3378381
epoch 31 0.27300838 0.2782908
epoch 41 0.2319678 0.2422182


In [16]:
class_preds = np.empty(pred.shape[0], dtype=np.int32)
for i,clas in enumerate(map(np.argmax, pred)):
    class_preds[i] = clas
    
class_preds

array([2, 0, 9, ..., 3, 9, 2])

In [66]:
TEMPPREDS = pd.DataFrame(class_preds)
TEMPPREDS.index = np.arange(1, len(TEMPPREDS)+1)
TEMPPREDS.reset_index(inplace = True)
TEMPPREDS = TEMPPREDS.rename({'index':'ImageId', TEMPPREDS.columns[1]:'Label'}, axis = 1)

In [73]:
TEMPPREDS.to_csv('TFNNpreds.csv', index = False)

In [68]:
PREDS = SeqNNpreds.argmax(axis=-1)
predcsv = pd.DataFrame(PREDS)
predcsv.index = np.arange(1, len(predcsv)+1)
predcsv.reset_index(inplace = True)
predcsv = predcsv.rename({'index':'ImageId', predcsv.columns[1]:'Label'}, axis = 1)

In [74]:
predcsv.to_csv('SeqNNpreds1.csv', index = False)

Final Scores: SeqNNpreds = .97114
              TFNNpreds = .93242
              
Note terrible accuracy scores but also not close enough to beat any of the other kaggle competitors.  Further development and I could probably achieve that 100% accuracy.